In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import time

from  keras.models import load_model
model = load_model('my_model.h5')
#model.summary()
model.load_weights('my_model_weights.h5')

#Using TensorFlow backend.

In [6]:
cap = cv2.VideoCapture(0)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH) # 取得影像的尺寸大小
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) # 取得影像的尺寸大小
print("Image Size: %d x %d" % (width, height)) #%d 以10 進位整數方式輸出

while(True):
    tStart = time.time() #計時開始
    ret, img_1 = cap.read()  # 從攝影機擷取一張影像
    img_2 = img_1[0:480,80:560]
    cv2.imshow('Camera',img_2)  # 顯示圖片
    
    img_2_gray = cv2.cvtColor(img_2, cv2.COLOR_BGR2GRAY)  # 將圖片轉為灰階
    img_3 = 255 - img_2_gray
    img_3 = img_3.astype('float32') #對資料型別進行轉換
    img_3_min = np.amin(img_3)
    img_4 = img_3 - np.amin(img_3) #一維數組a中的最小值，二維數組需通過axis指定行或列，獲取行或列的最小值 
    img_5 = 255 * img_4 / (np.amax(img_4))
    kernel = np.ones((5,5),np.uint8) #ˋ轉換1
    img_6 = cv2.dilate(img_5,kernel,iterations = 3) #第一個參數為二值化的影像， 第二個參數為使用的捲積 kernel，第三個參數為迭代次數(預設為1)
    img_7 = cv2.resize(img_6,(28,28),1) #把照片縮放 28*28
    img_8 = img_6.astype('uint8') #轉換 0 to 255
    cv2.imshow('input',img_8)

    #print(img_1.dtype, img_1.shape)
    #print(img_2.dtype, img_2.shape)

    x_test_image = np.reshape(img_7, (1,28,28))#轉換成28*28
    #print(np.amax(x_test_image))
    #print(np.amin(x_test_image))

    # convert 2-D 28x28 image to 4-D nx28x28x1  array
    x_Test4D=x_test_image.reshape(x_test_image.shape[0],28,28,1).astype('float32')
    # normalize the image numbers to 0~1
    x_Test4D_normalize = (x_Test4D / np.amax(x_test_image))
    #print(x_Test4D_normalize.shape)
    prediction=model.predict_classes(x_Test4D_normalize)


    filename = '%s%d%s' % ('./numbers/number_', prediction, '.jpg') #取的資料夾事先做的JPG
    img_9 = cv2.imread(filename)
    img_10 = cv2.resize(img_9,(480,480),1)
    cv2.imshow('inference',img_10)
    tEnd = time.time() #計時結束

    if cv2.waitKey(1) & 0xFF == ord('q'): #如果按Q即刻跳出
        cv2.imwrite('output.jpg', img_2)
        break

cap.release() # 釋放攝影機
cv2.destroyAllWindows() # 關閉所有 OpenCV 視窗

print('It takes %f sec for each frame' % (tEnd - tStart))

Image Size: 640 x 480
It takes 0.066194 sec for each frame
